In [1]:
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
from abc import ABC, abstractmethod
import json
import pyodbc
import pandas as pd
import requests

class Geradora:
    def __init__(self):
        self._data = None
    
    def ReadJson(self):
        with open("./data.json", 'r+') as file:
            self._data = json.load(file)
        return self._data

    def CreateEngine(self):
        data = self.ReadJson()
        conn_url = URL.create(
            "mssql+pyodbc",
            username=data['username'],
            password=data["password"],
            host=data["host"],
            port=1433,
            database=data["database"],
            query={
                "driver": "ODBC Driver 17 for SQL Server",
                "TrustServerCertificate": "yes"
            }
        )
        engine = create_engine(conn_url)
        return engine
    
    def create_query(self, query):
        try:
            dataframe = pd.read_sql_query(query, self.CreateEngine())
            return dataframe
        except Exception as error:
            print('Erro ao executar a consulta')
            print(error)
        

geradora = Geradora()
dataframe = pd.DataFrame(geradora.create_query("select * from ONE_Bank_Data"))
dataframe

,TopsAgentBankID,CustomerID,AccountName,Routing,StatementBalance
0,103,4448,AAB MM Reserve 8268,122105980,10192.63
1,104,4448,Alliance Association Bank OP,122105980,151991.69
2,105,4448,Pacific Western Checking 4807,122238200,0.00
3,106,4448,Alliance Association Bank OP,122105980,91090.93
4,107,4448,RBC Wealth Management,777777777,551427.49
...,...,...,...,...,...
85244,531,2836,Millcreek Townhomes by Sego Homes Assoc - MM,122105980,0.00
85245,282,4235,Alliance Operating Account,122401778,0.00
85246,283,4235,Alliance Reserve,122401778,0.00
85247,284,4235,Alliance Operating,122401778,0.00


In [ ]:
dataset = dataframe.iloc[:, 3].values

banks = {}
try:
    for i in dataset:
        url = f'https://www.routingnumbers.info/api/name.json?rn={i}'
        response = requests.get(url)
        data = json.loads(response.content)
        if data['message'] == 'OK':
            banks[i] = data['name']
        else:
            pass
except:
    pass

routing = []
bank_name = []
for i in banks:
    routing.append(i)
for i in banks.values():
    bank_name.append(i)
new_dataframe = pd.DataFrame(list(zip(bank_name, routing)), columns=[
                            'Bank', 'Routing'])

df1 = dataframe.merge(new_dataframe, on='Routing')